# RF6 Impact Attestation Metrics

Spec [here](https://plaid-cement-e44.notion.site/Impact-Attestations-Data-b8b1c79a624c46ad94305c74def64783).

In [1]:
import datetime
import json
import pandas as pd

from scripts.agora import agora_api
from scripts.farcaster import load_farcaster, refresh_farcaster
from scripts.metrics import calculate_metrics

# Load data

In [2]:
def convert_farcaster(farcaster_id):
    return int(farcaster_id['hex'], 16) if farcaster_id else None

In [3]:
#refresh_farcaster()
f = load_farcaster()

FARCASTER_USERS = pd.DataFrame(f)[['fid', 'username']].drop_duplicates().set_index('fid')['username'].to_dict()
FARCASTER_LINKED_ADDRESSES = pd.DataFrame(f)[['fid', 'address']].dropna().groupby('fid')['address'].apply(set).to_dict()
FARCASTER_ADDRESS_TO_ID = pd.DataFrame(f)[['fid', 'address']].dropna().set_index('address')['fid'].to_dict()

In [4]:
delegate_data = agora_api('delegates', params={'limit': 100, 'offset': 0, 'sort': 'most_delegators'})
rf6_projects = agora_api('retrofunding/rounds/6/projects')

DELEGATES = [x['address'].lower() for x in delegate_data]
AGORA_IDS = [x['projectId'] for x in rf6_projects]

Fetched a total of 100 items from delegates.
Fetched 100 items, offset now at 100
Fetched a total of 142 items from retrofunding/rounds/6/projects.


In [5]:
rf6_projects_data = [{
    'id': x['id'],
    'applicationId': x['applicationId'],
    'projectId': x['projectId'],
    'name': x['name'],
    'organization': x['organization']['name'] if x.get('organization') else None,
    'applicationCategory': x['applicationCategory']
    }
    for x in rf6_projects
]
rf6_projects_df = pd.DataFrame(rf6_projects_data)
AGORA_NAMES = rf6_projects_df.set_index('projectId')['name'].to_dict()
rf6_projects_df.to_csv('data/_local/rf6_projects.csv')

In [6]:
def load_json(json_path):
    with open(json_path, 'r') as f:
        data = json.load(f)
    return data

project_attestation_data = load_json('data/attestations/472.json')
print("Project attestations:", len(project_attestation_data))

app_attestation_data = load_json('data/attestations/609.json')
print("Application attestations:", len(app_attestation_data))

badgeholder_attestation_data = load_json('data/attestations/599.json')
print("Badgeholder attestations:", len(badgeholder_attestation_data))

gov_council_attestation_data = load_json('data/attestations/141.json')
print("Gov Council attestations:", len(gov_council_attestation_data))

impact_attestation_data = load_json('data/attestations/566.json')
print("Impact attestations:", len(impact_attestation_data))

Project attestations: 4321
Application attestations: 407
Badgeholder attestations: 305
Gov Council attestations: 222
Impact attestations: 793


In [7]:
ATTESTER_ADDR = "0xf6872d315cc2e1aff6abae5dd814fd54755fe97c"

df_project_metadata = pd.DataFrame(project_attestation_data)
df_project_metadata = df_project_metadata[df_project_metadata.attester.str.lower() == ATTESTER_ADDR]
df_project_metadata.sort_values(by="timeCreated", ascending=False, inplace=True)
df_project_metadata.drop_duplicates(subset="projectRefUID", keep="first", inplace=True)

PROJECTS = list(df_project_metadata['projectRefUID'].unique())
print("Projects:", len(PROJECTS))

Projects: 3070


In [8]:
df_applications = pd.DataFrame(app_attestation_data)
df_applications = df_applications[df_applications.attester.str.lower() == ATTESTER_ADDR]
df_applications['round'] = df_applications['round'].apply(lambda x: str(x))
df_applications.sort_values(by="timeCreated", ascending=False, inplace=True)
df_applications.drop_duplicates(subset="metadataSnapshotRefUID", keep="first", inplace=True)

latest_metadata = list(df_project_metadata.id.unique())
df_applications = df_applications[df_applications.metadataSnapshotRefUID.isin(latest_metadata)]
print("Applications:", len(df_applications))

Applications: 139


# Clean / filter attestation data

In [9]:
ATTESTERS = []

for a in badgeholder_attestation_data:
    if a['attester'] != '0xE4553b743E74dA3424Ac51f8C1E586fd43aE226F':
        continue
    if a['voterType'] == 'Guest':
        continue

    addr = a['recipient'].lower()
    fid = convert_farcaster(a['farcasterID'])
    username = FARCASTER_USERS.get(fid, '')
    ATTESTERS.append({
        'address': addr,
        'farcaster_id': fid,
        'farcaster_username': username,
        'governance_role': 'Citizen',
        'source': 'attestation_schema_599'
    })
    linked_addresses = FARCASTER_LINKED_ADDRESSES.get(fid, [])
    for addr in linked_addresses:
        ATTESTERS.append({
            'address': addr.lower(),
            'farcaster_id': fid,
            'farcaster_username': username,
            'governance_role': 'Citizen',
            'source': 'farcaster_linked_address'
        })

for addr in DELEGATES:
    fid = FARCASTER_ADDRESS_TO_ID.get(addr, '')
    username = FARCASTER_USERS.get(fid, '')
    ATTESTERS.append({
        'address': addr.lower(),
        'farcaster_id': fid,
        'farcaster_username': username,
        'governance_role': 'Top 100 Delegate',
        'source': 'agora_api'
    })
    linked_addresses = FARCASTER_LINKED_ADDRESSES.get(fid, [])
    for addr in linked_addresses:
        ATTESTERS.append({
            'address': addr.lower(),
            'farcaster_id': fid,
            'farcaster_username': username,
            'governance_role': 'Top 100 Delegate',
            'source': 'farcaster_linked_address'
        })
    
len(ATTESTERS)

560

In [10]:
mapped_roles = {
    'ACC (Second half)': 'Anticapture Commission',
    'Anticapture Commission Member': 'Anticapture Commission',
    'Badgeholder Reviewer': None,
    'Code of Conduct Council': 'Code of Conduct',
    'Code of Conduct Council Lead': 'Code of Conduct',
    'Code of Conduct Council Member': 'Code of Conduct',
    'Developer Advisory Board Lead': 'Developer Advisory Board',
    'Developer Advisory Board Member': 'Developer Advisory Board',
    'Optimism Grants Council Lead': 'Grants Council',
    'Optimism Grants Council Member': 'Grants Council',
    'Security Council Lead': 'Security Council',
    'Security Council Member': 'Security Council',
}
df_gov_councils = pd.DataFrame(gov_council_attestation_data)
df_gov_councils = df_gov_councils[
    (df_gov_councils['attester'] == '0xE4553b743E74dA3424Ac51f8C1E586fd43aE226F')
    & (df_gov_councils['govSeason'].isin(['5','6']))
]
df_gov_councils['recipient'] = df_gov_councils['recipient'].str.lower()
df_gov_councils['governance_council'] = df_gov_councils['govRole'].map(mapped_roles)

In [11]:
df_attesters = (
    pd.DataFrame(ATTESTERS)
    .groupby(['address', 'farcaster_id', 'farcaster_username'])['governance_role']
    .agg(set)
    .reset_index()
    .set_index('address')
    .join(
        df_gov_councils
        .groupby('recipient')['governance_council']
        .apply(set)
    )
    .sort_values(by='governance_council')
    .reset_index()
    .rename(columns={'index': 'address'})
)

df_attesters.to_csv("data/_local/governance_users.csv")
df_attesters

,address,farcaster_id,farcaster_username,governance_role,governance_council
0,0x07fda67513ec0897866098a11dc3858089d4a505,318387,v3naru,{Citizen},{Grants Council}
1,0xe93d59cc0bcecfd4ac204827ef67c5266079e2b5,,,{Top 100 Delegate},{Anticapture Commission}
2,0xe422d6c46a69e989ba6468ccd0435cb0c5c243e3,195722,joanbp,{Citizen},{None}
3,0xdc0a92c350a52b6583e235a57901b8731af8b249,247606,cpstl,{Citizen},{None}
4,0x8c580556fdb1f57853e49f409ae9b89f7658e7a2,7255,wish,{Top 100 Delegate},"{Anticapture Commission, nan}"
...,...,...,...,...,...
227,0xfb0a98121c3939657e59f835c66cfe20abb0cd4f,233135,krzkaczor,{Citizen},NaN
228,0xfb9494c0f9ca9e8371a7c744e4c5343708989f0b,555729,lubiyangjia,{Top 100 Delegate},NaN
229,0xfbe82589349841130791225d1756bd05157eb560,573324,luckyrousong,{Top 100 Delegate},NaN
230,0xfc61965861b679c5aa728d41fa6ea0b29544f554,,,{Top 100 Delegate},NaN


In [12]:
df = pd.DataFrame(impact_attestation_data)
df['farcasterID'] = df['farcasterID'].apply(convert_farcaster)
df['farcasterUsername'] = df['farcasterID'].map(FARCASTER_USERS)
df['timeCreated'] = df['timeCreated'].apply(lambda x: datetime.datetime.utcfromtimestamp(x))

def get_name(metadata):
    if metadata.get('projectName'):
        return metadata.get('projectName')
    elif metadata.get('project'):
        return metadata.get('project').get('name')
    else:
        return None

df['mgl_project_name'] = df['metadata'].apply(get_name)

# provided by MGL team
overrides = {
    "0x8baac027df9c78c3dbda664c8f0190391e6b4bdabf240da9ccc4f0837933d8f2":"0x7753d360506e1d538e9a4720677595b2b811b3a3e74b81bd54c52ddfaaf51537",
    "0xfbca8cf7af5e74ffe1aea1f6850d0a427fb77286db170e478aca89f09b68e57c":"0xbb1e4aadda991e93d4ba4630582f222b2c57c0b0d729e29247cf2a3873936808",
    "0xfbca8cf7af5e74ffe1aea1f6850d0a427fb77286db170e478aca89f09b68e57c":"0xc750e06098f5e1e25a1535c8b9237f258777455466a53bb392738ba4bb7799c4"
}
df['agoraProjectRegUID'] = df['projectRegUID'].apply(lambda x: overrides.get(x,x))

df['agora_project_name'] = df['agoraProjectRegUID'].map(AGORA_NAMES)

def get_scores(metadata):
    impact_eval_questions = metadata.get('impactAttestations', [])
    nps = pmf = None
    for q in impact_eval_questions:
        val = q.get('value')
        if not val:
            continue
        if q['name'] == 'Likely to Recommend':
            nps = pd.to_numeric(val)
        elif q['name'] == 'Feeling if didnt exist':
            pmf = pd.to_numeric(val)
    return {
        "NPS": nps,
        "PMF": pmf
    }

df['nps_score'] = df['metadata'].apply(lambda x: get_scores(x)['NPS'])
df['pmf_score'] = df['metadata'].apply(lambda x: get_scores(x)['PMF'])

roles = df_attesters.groupby('farcaster_id')['governance_role'].agg(lambda x: set.union(*x)).to_dict()
councils = (
    df_attesters[['farcaster_id', 'governance_council']]
    .dropna()
    .groupby('farcaster_id')['governance_council']
    .agg(lambda x: set.union(*x)).to_dict()
)
councils = {key: value for key, value in councils.items() if value != {None} and key != ''}
df['governance_role'] = df['farcasterID'].map(roles)
df['is_citizen'] = df['governance_role'].apply(lambda x: isinstance(x, set) and 'Citizen' in x)
df['is_top_delegate'] = df['governance_role'].apply(lambda x: isinstance(x, set) and 'Top 100 Delegate' in x)
df['governance_membership'] = df['farcasterID'].map(councils)

df.head()

,id,attester,recipient,timeCreated,contributionRegUID,projectRegUID,farcasterID,issuer,metadataurl,metadata,farcasterUsername,mgl_project_name,agoraProjectRegUID,agora_project_name,nps_score,pmf_score,governance_role,is_citizen,is_top_delegate,governance_membership
0,0x027e4643bcc39307d567db9f95e27bab8856634defab...,0x19f8a76474EbF9EFFB269ec5C2B935A3611D6779,0x0000000000000000000000000000000000000000,2024-09-07 16:27:13,0x28a18006a584489339f6d5266a7585f1c8cbabcb89b2...,0x28a18006a584489339f6d5266a7585f1c8cbabcb89b2...,4155,MGL,https://gateway.pinata.cloud/ipfs/Qmad5wK1PQbz...,"{'id': 1683, 'userFid': '5882', 'projectName':...",tynes,Fault Dispute Game Audit Contest,0x28a18006a584489339f6d5266a7585f1c8cbabcb89b2...,NaN,NaN,NaN,NaN,False,False,NaN
1,0x02bddaaec00e7ea06da0ad12ed7a4d33b76318903af9...,0x7484aABFef9f39464F332e632047983b67571C0a,0x5d36a202687fD6Bd0f670545334bF0B4827Cc1E2,2024-09-20 19:28:32,0xe86fad0dd8e3bfe0bfe7c19940f6d21685e3a67256bb...,0x82169691e4026be1d950f20c2d36500b06fb6f3e0c03...,429828,MGL,https://gateway.pinata.cloud/ipfs/QmRswtwabhoW...,"{'id': 1680, 'userFid': '11596', 'projectName'...",niallinio,Ethereum Mexico PGF,0x82169691e4026be1d950f20c2d36500b06fb6f3e0c03...,NaN,NaN,NaN,NaN,False,False,NaN
2,0x03fe95d59be66b69538e9840df229b1fff540f234c1e...,0x7484aABFef9f39464F332e632047983b67571C0a,0x5d36a202687fD6Bd0f670545334bF0B4827Cc1E2,2024-10-01 11:13:11,0xe86fad0dd8e3bfe0bfe7c19940f6d21685e3a67256bb...,0x82169691e4026be1d950f20c2d36500b06fb6f3e0c03...,429828,MGL,https://gateway.pinata.cloud/ipfs/QmesYEayK4Sj...,"{'id': 1680, 'project': {'name': 'Ethereum Mex...",niallinio,Ethereum Mexico PGF,0x82169691e4026be1d950f20c2d36500b06fb6f3e0c03...,NaN,NaN,NaN,NaN,False,False,NaN
3,0x07325b942e18dce9469543e0cfa3d95813907a15e596...,0x7484aABFef9f39464F332e632047983b67571C0a,0xEdD95782d13902Ae535332b5D233041E47aD69E6,2024-09-24 22:17:47,0x60d53d3890856123b9736ee8f6ef9c96ce8a218ec1db...,0xfbca8cf7af5e74ffe1aea1f6850d0a427fb77286db17...,9630,MGL,https://gateway.pinata.cloud/ipfs/QmaCq3mQ1CSA...,"{'id': 1777, 'userFid': '350951', 'projectName...",chaselb,Tribuni,0xc750e06098f5e1e25a1535c8b9237f258777455466a5...,OP Bulletin,NaN,NaN,NaN,False,False,NaN
4,0x097cdb0c05591224259ca98ca7a8a5b2c3edf2845ed0...,0x7484aABFef9f39464F332e632047983b67571C0a,0x0000000000000000000000000000000000000000,2024-09-26 00:01:22,0xa2e237bf8681844563ac374f3ffa599945e78ac6b599...,0xf358fd1a0dabf82630a1e3acd512626a6e4677b4e658...,10066,MGL,https://gateway.pinata.cloud/ipfs/QmQJZrPw332K...,"{'id': 1779, 'userFid': '436039', 'projectName...",monitalan,Web3 Citizen,0xf358fd1a0dabf82630a1e3acd512626a6e4677b4e658...,NaN,NaN,NaN,NaN,False,False,NaN


# Create attestations dataframe

In [13]:
valid_attestations = (
    (df['agora_project_name'].isna()==False)
    & (df['governance_role'].isna()==False)
    & (df['nps_score'].isna() == False)
)
df_filtered = df[valid_attestations]
df_filtered['agora_project_name'].value_counts()

agora_project_name
Impact Evaluation Framework + Metrics Garden Database                               20
Snapshot                                                                            18
Pairwise                                                                            12
Superfluid                                                                          10
OP Governance App                                                                   10
CharmVerse Grants                                                                    9
RetroList                                                                            8
Curia OP Governance Analytics Dashboard                                              8
RetroPGF Hub                                                                         8
Gitcoin                                                                              8
Retro Funding Application Review                                                     7
Retro Funding 3 Voting A

In [14]:
df_normalized = df.copy()

df_normalized['valid_attestation'] = valid_attestations

cols = [
    'timeCreated', 'projectRegUID', 'mgl_project_name', 'agoraProjectRegUID', 'agora_project_name',
    'farcasterUsername', 'farcasterID', 'is_citizen', 'is_top_delegate', 'governance_role',
    'governance_membership', 'nps_score', 'pmf_score', 'valid_attestation',
    'id', 'attester', 'recipient', 'contributionRegUID', 'metadataurl', 'metadata'
]

df_normalized = df_normalized[cols]
df_normalized.sort_values(by='timeCreated', ascending=False, inplace=True)
df_normalized.to_csv("data/normalized_attestations.csv")  

# Implement metrics on relevant attestations

In [15]:
metrics = calculate_metrics(df_filtered)

with open("data/attestation_metrics.json", "w") as f:
    json.dump(metrics,f,indent=2)

# Create dummy attestations

In [16]:
import random
import secrets

In [17]:
ATTESTERS = {}
for _, a in df_attesters.iterrows():
    addr = a['address']
    if addr in ATTESTERS:
        continue
    ATTESTERS.update({
        addr: {
            'is_citizen': 'Citizen' in a['governance_role'],
            'is_top_delegate': 'Top 100 Delegate' in a['governance_role'],
            'governance_membership': a['governance_council'] if isinstance(a, set) else {}
        }
    })

In [18]:
NUM_PROJECTS = 30
NUM_ATTESTERS = 100
NUM_ATTESTATIONS = 1000
PMF_SCALE = 3
NPS_SCALE = 10

projects = {
    p: random.random()
    for p in random.choices(PROJECTS, k=NUM_PROJECTS)
}

attesters = {
    addr:ATTESTERS[addr]
    for addr in random.choices(list(ATTESTERS.keys()), k=NUM_ATTESTERS)
}

dummy_attestations = []
for i in range(NUM_ATTESTATIONS):
    random_project = random.choice(list(projects.keys()))
    random_attester = random.choice(list(attesters.keys()))
    project_impact = projects.get(random_project)
    pmf_score = random.randint(int(project_impact*PMF_SCALE), PMF_SCALE)
    nps_score = random.randint(int(project_impact*NPS_SCALE), NPS_SCALE)
    dummy_attestations.append({
        'id': f"0x{secrets.token_hex(32)}",
        'attester': random_attester,
        'projectRegUID': random_project,
        'pmf_score': max(1,pmf_score),
        'nps_score': max(1,nps_score),
        **attesters[random_attester]
    })

In [19]:
df_dummy = pd.DataFrame(dummy_attestations)
dummy_metrics = calculate_metrics(df_dummy)

In [20]:
with open("data/dummy_metrics_v2.json", "w") as f:
    json.dump(dummy_metrics, f, indent=2)